# BB84 Protocol - QBER and SKR Definitions and Analysis

## Introduction to BB84 Performance Metrics

The BB84 quantum key distribution protocol's performance is fundamentally characterized by two critical metrics: the Quantum Bit Error Rate (QBER) and the Secret Key Rate (SKR). These parameters determine the security and practical viability of quantum cryptographic systems across different transmission media.

## Quantum Bit Error Rate (QBER) - Definition and Theory

### QBER Definition

The Quantum Bit Error Rate (QBER) quantifies the proportion of erroneous bits in the quantum channel transmission. For the BB84 protocol, QBER is mathematically defined as:

```
QBER = (Number of Error Events) / (Total Detection Events)
```

### QBER Formula for BB84 Protocol

For practical BB84 implementations using weak coherent pulses (WCP) with decoy states, the QBER is expressed as:

```
QBER_BB84 = (c × p_signal + 0.5 × (p_dark + p_straycounts)) / p_click
```

**Where:**
- **c**: Intrinsic channel error rate (typically 0.01-0.05 for high-quality systems)
- **p_signal**: Signal detection probability = 1 - exp(-μ × η_total)
- **p_dark**: Dark count probability per pulse (detector noise)
- **p_stray**: Probability of stray photons or background light triggering the detector
- **p_click**: Total click probability = p_signal + p_dark + p_straycounts

### Physical Interpretation

The QBER formula captures two primary error sources:
1. **Intrinsic channel errors**: Represented by the term `c × p_signal`, arising from phase fluctuations, polarization drift, and environmental disturbances
2. **Dark count contributions**: The term `0.5 × p_dark` accounts for random detector clicks, where 50% probability assumes random bit assignment for dark counts
3. **Stray photon contributions**: The term `0.5 × p_straycounts` models clicks from background light or stray photons, treated as random with 50% error probability

## SKR Formula for BB84 Protocol

In a more realistic setting (with multiphoton pulses, dark counts, etc.), the **SKR for BB84** is computed as:

$$
\text{SKR}_{\text{BB84}} = \frac{1}{2} \cdot p_{\text{click}} \cdot \left[ \beta \cdot \tau\left( \frac{e}{\beta} \right) - f_e \cdot H(e) \right]
$$

**Where:**

- $ \frac{1}{2} $: Accounts for basis reconciliation loss (only ~50% of bits are usable).
- $ p_{\text{click}} $: Total probability that a detector registers a signal.
- $ \beta $: Fraction of detections due to **single-photon events** (non-multiphoton).
- $ \tau(e / \beta) $: **Privacy amplification** term — reduces eavesdropper’s knowledge.
- $ f_e $: **Error correction inefficiency**, typically $ f_e = 1.16 $.
- $ H(e) $: Binary entropy function:

$$
H(e) = -e \log_2(e) - (1 - e) \log_2(1 - e)
$$


---

#### Supporting Definitions

- **Click Probability:**

  $$
  p_{\text{click}} = p_{\text{signal}} + p_{\text{dark}} + p_{\text{stray}}
  $$

- **QBER (Error Rate):**

  $$
  e = \frac{c_{\text{error}} \cdot p_{\text{signal}} + 0.5 \cdot (p_{\text{dark}} + p_{\text{stray}})}{p_{\text{click}}}
  $$

- **Multi-photon Approximation (for BB84 with weak coherent pulses):**

  $$
  p_{\text{multi}}(\mu) = 1 - \left(1 + \mu + \frac{\mu^2}{2} + \frac{\mu^3}{12}\right)e^{-\mu}
  $$

- **Single-photon fraction \( \beta \):**

  $$
  \beta = \frac{p_{\text{click}} - p_{\text{multi}}(\mu)}{p_{\text{click}}}
  $$

- **Privacy Amplification Function:**

  $$
  \tau\left(\frac{e}{\beta}\right) = 
  \begin{cases}
    \log_2\left(1 + 4 \cdot \frac{e}{\beta} - 4 \cdot \left( \frac{e}{\beta} \right)^2\right), & \text{if } \frac{e}{\beta} < 0.5 \\\\
    1, & \text{otherwise}
  \end{cases}
  $$


This formula tightly bounds the SKR by penalizing multiphoton emissions and high QBER values while accurately reflecting physical-layer imperfections.


### Security Threshold

The BB84 protocol maintains unconditional security only when QBER < 11%. Beyond this threshold:
- Privacy amplification cannot generate secure keys
- The channel becomes vulnerable to eavesdropping attacks
- SKR approaches zero asymptotically

## Parameter Definitions and Physical Significance

### System Parameters

| Parameter | Symbol | Typical Range | Physical Meaning |
|-----------|--------|---------------|------------------|
| Detector Efficiency | η_det | 0.1 - 0.9 | Probability of photon detection at receiver |
| Mean Photon Number | μ | 0.1 - 1.0 | Average photons per pulse (WCP assumption) |
| Intrinsic Error Rate | c | 0.01 - 0.05 | Channel-induced bit errors |
| Dark Count Rate | d | 10⁻⁶ - 10⁻⁸ | False detection probability per pulse |
| Error Correction Factor | f_e | 1.16 - 1.5 | Overhead for practical error correction |


## Free Space Optical (FSO) Channel Analysis

### FSO System Parameters

**Physical Parameters:**
- **Receiver aperture diameter (d_r)**: Typically 0.1-1.0 m, determines light collection capability
- **Transmitter aperture diameter (d_t)**: Typically 0.05-0.5 m, affects initial beam size
- **Beam divergence (θ)**: Typically 0.1-2.0 mrad, determines beam spreading rate
- **Atmospheric attenuation (α)**: 0.1-2.0 dB/km, highly weather-dependent

### FSO Transmission Efficiency Model

The FSO channel transmission efficiency combines two primary loss mechanisms:

```
η_T^FSO(L) = η_geometric(L) × η_atmospheric(L)
```

**Geometric Spreading Loss:**
As the laser beam propagates through free space, it spreads according to diffraction limits and beam divergence:

```
η_geometric(L) = (d_r / (d_t + θ × L))²
```

This quadratic distance dependence represents the fundamental challenge of FSO links - beam spreading reduces power density at the receiver.

**Atmospheric Attenuation:**
Photons are absorbed and scattered by atmospheric particles:

```
η_atmospheric(L) = exp(-α × L)
```

Where α is converted from dB/km to natural units: `α = α_dB / (10/ln(10)) / 1000`

**Combined FSO Model:**
```
η_T^FSO(L) = (d_r / (d_t + θ × L))² × exp(-α_dB × L / 4343)
```

### FSO Channel Characteristics

**Weather Dependencies:**
- **Clear conditions**: α ≈ 0.1-0.5 dB/km
- **Haze/light fog**: α ≈ 2-10 dB/km  
- **Dense fog**: α > 50 dB/km (link becomes impractical)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Common Constants ===================
eta_det = 0.6              # Detector efficiency
d = 8e-6                   # Dark count probability
c = 0.01                   # Intrinsic error rate
alpha_db = 0.5             # Atmospheric attenuation (dB/km)
alpha = alpha_db / (4.343*1000)   # Convert dB/km to 1/m
d_t = 0.2                  # Transmitter aperture diameter (m)
d_r = 0.2                  # Receiver aperture diameter (m)
D = 0.3e-3                 # Beam divergence (radians)
f_e_bbm92 = 1.16           # Error correction inefficiency factor (BBM92)
f_ec_bb84 = 1.16           # Error correction inefficiency factor (BB84)

# Distance values ==================

L_km = np.linspace(0.1, 50, 500)
L_m = L_km * 1000

# BBM92 Calculations ===================

geometric_loss = (d_r / (d_t + D * L_m))**2
atmospheric_loss = np.exp(-alpha * L_m)
eta_T_bbm92 = geometric_loss * atmospheric_loss
alpha_L = eta_det * eta_T_bbm92
alpha_x = np.sqrt(alpha_L)

p_true = eta_det * alpha_L
p_false = 8 * alpha_x * d + 16 * d**2
p_straycounts = 5e-6
p_coin = p_true + p_false + p_straycounts

e_bbm92 = (c * p_true + 0.5 * (p_false + p_straycounts)) / p_coin

def tau_bbm92(e):
    val = 0.5 + 2 * e - 2 * e**2
    val = np.clip(val, 1e-12, 1)  # avoid log2(0) or negative
    return np.where(e < 0.5, -np.log2(val), 0)


def binary_entropy(x):
    x = np.clip(x, 1e-12, 1 - 1e-12)
    return -x * np.log2(x) - (1 - x) * np.log2(1 - x)

skr_bbm92 = p_coin * 0.5 * (tau_bbm92(e_bbm92) - f_e_bbm92 * binary_entropy(e_bbm92))

# BB84 Calculations ===================

mu = 0.5
c_error = 0.02

def eta_t(L):  # L in meters
    denom = d_t + D * L
    geom = (d_r / denom) ** 2
    atmos = np.exp(-alpha * L)
    return geom * atmos

def p_stray():
    return 5e-6

def p_dark():
    return 4e-8

def p_signal_bb84(eta):
    return 1 - np.exp(-eta_det * eta * mu)

def p_prime(mu):
    return 1 - (1 + mu + mu*2 / 2 + mu*3 / 12) * np.exp(-mu)

def tau_bb84(e):
    return -1*np.log2(0.5 + 2 * e - 2 * e**2) if e < 0.5 else 0

def f_e(e): return f_ec_bb84

def QBER_BB84(eta):
    signal = p_signal_bb84(eta)
    stray = p_stray()
    dark = p_dark()
    return (c_error * signal + 0.5 * (dark + stray)) / (signal + dark + stray)

def beta(p_click_val):
    p_p = p_prime(mu)
    result = (p_click_val - p_p) / p_click_val
    return np.clip(result, 1e-6, 1)

def R_BB84(eta):
    signal = p_signal_bb84(eta)
    stray = p_stray()
    dark = p_dark()
    p_click = signal + dark + stray
    e = QBER_BB84(eta)
    e = np.clip(e, 1e-12, 1 - 1e-12)
    
    beta_val = beta(p_click)
    e_div_beta = np.clip(e / beta_val, 1e-12, 1 - 1e-12)
    tau_val = np.vectorize(tau_bb84)(e_div_beta)
    
    h_e = -e * np.log2(e) - (1 - e) * np.log2(1 - e)
    skr = 0.5 * p_click * (beta_val * tau_val - f_e(e) * h_e)
    skr[e > 0.11] = 0
    skr[skr < 0] = 0
    return skr

eta_vals_bb84 = eta_t(L_m)
qber_bb84 = QBER_BB84(eta_vals_bb84)
skr_bb84 = R_BB84(eta_vals_bb84)

# Plot QBER ===================

plt.figure(figsize=(10, 5))
plt.plot(L_km, e_bbm92, label='QBER (BBM92)', color='tab:blue')
plt.plot(L_km, qber_bb84, label='QBER (BB84)', color='tab:purple', linestyle='--')
plt.xlabel("FSO Link Distance (km)")
plt.ylabel("Quantum Bit Error Rate (QBER)")
plt.title("QBER vs Distance for BB84 and BBM92 (FSO)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Plot SKR ===================

plt.figure(figsize=(10, 5))
plt.plot(L_km, skr_bbm92, label='SKR (BBM92)', color='tab:green')
plt.plot(L_km, skr_bb84, label='SKR (BB84)', color='tab:orange', linestyle='--')
plt.xlabel("FSO Link Distance (km)")
plt.ylabel("Secret Key Rate (bits/pulse)")
plt.title("Secret Key Rate vs Distance for BB84 and BBM92 (FSO)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


## Optical Fiber Channel Analysis

### Fiber System Parameters

**Key Parameters:**
- **Fiber attenuation coefficient (α_fiber)**: 0.15-0.35 dB/km depending on wavelength and fiber type
- **Coupling efficiency (η_coupling)**: 0.5-0.9, accounts for splice and connector losses

### Fiber Transmission Efficiency Model

Optical fiber transmission follows Beer's law with additional coupling considerations:

```
η_T^Fiber(L) = η_coupling × exp(-α_fiber × L /10)
```

**Wavelength Optimization:**
- **1310 nm window**: ~0.35 dB/km, zero dispersion wavelength
- **1550 nm window**: ~0.2 dB/km, minimum attenuation (optimal for long-distance QKD)

### Fiber Channel Characteristics  

### FSO vs Fiber Comparison

| Aspect | FSO | Optical Fiber |
|--------|-----|---------------|
| **Loss Scaling** | L⁻² (geometric) + exp(-αL) | exp(-αL) only |
| **Weather Sensitivity** | High (fog, rain critical) | None (protected) |
| **Deployment** | Line-of-sight required | Infrastructure needed |

This analysis provides the theoretical foundation for understanding quantum channel performance across different transmission media, enabling informed system design decisions based on specific deployment requirements.